In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import datetime
import random
rs = 420
random.seed(rs)

print(datetime.datetime.today().strftime("%HH:%MM %dd"))
# Load datasets


vessel_encoder = LabelEncoder()

# Select features and target
features = ['vesselId', 'hour', 'day', 'month', 'year', 'minute', 'yearBuilt', 'length', 'vesselType', 'breadth', 'CEU', 'DWT', 'GT', 'portLatitude', 'portLongitude', 'sch_hour', 'sch_minute']#['cog', 'sog', 'rot', 'heading', 'navstat', 'latitude', 'longitude', 'vesselId', 'portId', 'hour', 'day', 'month', 'year', 'minute']
target = ['latitude', 'longitude']  # Predicting next position (latitude, longitude)

def feature_engineering(data):

    data['vesselType'].fillna(83, inplace=True) #data['vesselType'].mode() -> dette er 83 men funker ikke??
    data['yearBuilt'].fillna(data['yearBuilt'].median(), inplace=True)
    data['length'].fillna(data['length'].median(), inplace=True)
    data['breadth'].fillna(data['breadth'].median(), inplace=True)
    data['CEU'].fillna(data['CEU'].median(), inplace=True)
    data['DWT'].fillna(data['DWT'].median(), inplace=True)
    data['GT'].fillna(data['GT'].median(), inplace=True)
    data['portLatitude'].fillna(0, inplace=True)
    data['portLongitude'].fillna(0, inplace=True)
    data['sch_hour'].fillna(0, inplace=True)
    data['sch_minute'].fillna(0, inplace=True)
    return data

# Handle missing values (if any)
#ais_train = ais_train.dropna(subset=features + target) #ais_train.replace(to_replace='None', value=np.nan).dropna()


# Train-test split
X_0 = pd.read_csv('x_data.csv', sep=',')
X_1 = pd.read_csv('x_data_1.csv', sep=',')
X_2 = pd.read_csv('x_data_2.csv', sep=',')
X_3 = pd.read_csv('x_data_3.csv', sep=',')
X = X_0 + X_1 + X_2 + X_3
y = pd.read_csv('y_data.csv', sep=',')

X['vesselId'] = vessel_encoder.fit_transform(X['vesselId'])

model = RandomForestRegressor(n_estimators=1000, random_state=rs)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=rs)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Train a RandomForest model
model.fit(X_train_scaled, y_train)

# Predict on validation set
y_pred = model.predict(X_val_scaled)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Absolute Error: {mse}')
print(f'Root Mean Squared Error: {np.sqrt(mse)}')
print(f'R2-score: {r2_score(y_val, y_pred)}')
print(f'Variance Score: {explained_variance_score(y_val, y_pred)}')

# Use the same features as training
X_0 = pd.read_csv('x_test.csv', sep=',')
X_1 = pd.read_csv('x_test_1.csv', sep=',')
X_2 = pd.read_csv('x_test_2.csv', sep=',')
X_3 = pd.read_csv('x_test_3.csv', sep=',')
X_test = X_0 + X_1 + X_2 + X_3
X_test['vesselId'] = vessel_encoder.transform(X_test['vesselId'])
# Scale the test set
X_test_scaled = scaler.transform(X_test)

# Predict on the test set
test_predictions = model.predict(X_test_scaled)

# Save the predictions
predictions_df = pd.DataFrame(test_predictions, columns=['longitude_predicted', 'latitude_predicted'])
predictions_df.to_csv('ais_test_predictions.csv', index=True)
print(datetime.datetime.today().strftime("%HH:%MM %dd"))